In [1]:
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import torchvision

import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn 
import torch

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <11CA4EBF-5A84-3662-A35A-A039CEE97F5E> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset   = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset    = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

In [ ]:
device = torch.device("mps:0")
print(device)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3),                nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 192, 3),              nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(192, 384, 3, padding=1),  nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(384, 256, 3, padding=1),  nn.ReLU(),
            nn.Conv2d(256, 256, 1),             nn.ReLU(), nn.MaxPool2d(2, 2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 3 * 3, 1024),       nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),               nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 8)
        x = self.classifier(x)
        return x

In [ ]:
criterion = nn.CrossEntropyLoss()
alexnet = AlexNet().to(device)
optimizer = optim.Adam(alexnet.parameters(), lr=1e-3)

In [ ]:
loss = list()
n = len(trainloader)

for epoch in range(50):
    running_loss = 0.0
    for data in trainloader:
        inputs, labels = data[0].to(device), data[1].to(device)

        # Forward
        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)

        # Backward
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    loss.append(running_loss / n)
    print(f"{epoch + 1} loss : {running_loss / len(trainloader)}")
